In [2]:
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.window import Window
from sklearn import preprocessing # https://github.com/Snowflake-Labs/snowpark-python-demos/tree/main/sp4py_utilities
from snowflake.snowpark.functions import col

import getpass
import pandas as pd
import matplotlib.pyplot as plt
import decimal
import joblib 

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
accountname =  "ex39201.ap-southeast-1"# ORGNAME-ACCOUNTNAME (separated by minus sign)
username = "Vibu"    # SNOWFLAKE-USERNAME
password = "s10222756B"   # SNOWFLAKE-PASSWORD

In [4]:
connection_parameters = {
    "account": accountname,
    "user": username,
    "password": password,
    "role": "ACCOUNTADMIN",
    "database": "FROSTBYTE_TASTY_BYTES",

    "warehouse": "TASTY_DE_WH"
}

session = Session.builder.configs(connection_parameters).create()

In [26]:
to_encode_df = session.table('ANALYTICS.SALES_PREDICTION')

In [38]:
columns_to_drop = ['"r_uwp5_LOCATION_ID"','"l_99ts_COUNTRY"','"l_w5i8_POSTAL_CODE"','"r_a073_POSTAL_CODE"','"r_a073_COUNTRY"','"r_a073_DATE"'] #'r_ipsj' keys changes everytime file is run
new_columns  = [c for c in to_encode_df.columns if c not in columns_to_drop]
dropped_df = to_encode_df.select(*new_columns)

In [39]:
dropped_df.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"l_w5i8_DATE"  |"TRUCK_ID"  |"MONTH"  |"HOUR"  |"DOW"  |"DAY"  |"MENU_TYPE"  |"l_c6x2_LOCATION_ID"  |"PUBLIC_HOLIDAY"  |"SUM(ORDER_TOTAL)"  |"LOCATION"         |"CITY"         |"REGION"  |"l_w5i8_COUNTRY"  |"LAT"               |"LONG"               |"AVG_TEMPERATURE_AIR_2M_F"  |"AVG_TEMPERATURE_HEATINDEX_2M_F"  |"TOT_PRECIPITATION_IN"  |"TOT_SNOWFALL_IN"  |"TOT_SNOWDEPTH_IN"  |"AVG_CLOUD_COVER_TOT_PCT"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [60]:
mdf=session.sql("Select * from ANALYTICS.DEMAND_FORECAST_TRAINING_BASE")
merge_df=dropped_df.join(mdf, (dropped_df['"l_w5i8_DATE"'] == mdf["DATE"])& (dropped_df['"MENU_TYPE"'] == mdf["MENU_TYPE"])& (dropped_df['"l_c6x2_LOCATION_ID"'] == mdf["Location_ID"])).drop('"l_w5i8_DATE"').drop('"l_3iuv_LOCATION_ID"')


In [55]:
merge_df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"TRUCK_ID"  |"MONTH"  |"HOUR"  |"DOW"  |"DAY"  |"l_mf63_MENU_TYPE"  |"l_c6x2_LOCATION_ID"  |"PUBLIC_HOLIDAY"  |"SUM(ORDER_TOTAL)"  |"LOCATION"         |"CITY"         |"REGION"  |"l_w5i8_COUNTRY"  |"LAT"               |"LONG"               |"AVG_TEMPERATURE_AIR_2M_F"  |"AVG_TEMPERATURE_HEATINDEX_2M_F"  |"TOT_PRECIPITATION_IN"  |"TOT_SNOWFALL_IN"  |"TOT_SNOWDEPTH_IN"  |"AVG_CLOUD_COVER_TOT_PCT"  |"DATE"      |"LOCATION_ID"  |"r_ttg7_MENU_TYPE"  |"SUM_DAY_OF_WEE

In [61]:
merge_df=merge_df.drop('"l_w5i8_COUNTRY"')
merge_df = merge_df.drop('"l_c6x2_LOCATION_ID"')

In [62]:
merge_df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"TRUCK_ID"  |"MONTH"  |"HOUR"  |"DOW"  |"DAY"  |"l_aqo3_MENU_TYPE"  |"PUBLIC_HOLIDAY"  |"SUM(ORDER_TOTAL)"  |"LOCATION"         |"CITY"         |"REGION"  |"LAT"               |"LONG"               |"AVG_TEMPERATURE_AIR_2M_F"  |"AVG_TEMPERATURE_HEATINDEX_2M_F"  |"TOT_PRECIPITATION_IN"  |"TOT_SNOWFALL_IN"  |"TOT_SNOWDEPTH_IN"  |"AVG_CLOUD_COVER_TOT_PCT"  |"DATE"      |"LOCATION_ID"  |"r_mncm_MENU_TYPE"  |"SUM_DAY_OF_WEEK_AVG_CITY_MENU_TYPE"  |"SUM_PREV_YEAR_MONTH_SALES_CITY_MENU_TYPE"  |
--------------

In [63]:

merge_df=merge_df.drop('"r_mncm_MENU_TYPE"')
merge_df = merge_df.withColumnRenamed('"l_aqo3_MENU_TYPE"','Menu_Type')




In [67]:
merge_df=merge_df.with_column("YEAR",F.year(merge_df["DATE"]))

In [68]:
wdf=session.sql("Select * from ANALYTICS.WEATHER_DATA_API")
wdf=wdf.withColumn("H",F.substring(wdf["TIME"], 12, 2).cast("integer"))
wdf=wdf.withColumn("DATE",F.substring(wdf["TIME"], 0, 10))
wdf=wdf.select("WEATHERCODE","LOCATION_ID","H","DATE" )

In [70]:
merge_df_api=merge_df.join(wdf,(merge_df["DATE"]==wdf["DATE"])&(merge_df["LOCATION_ID"]==wdf["LOCATION_ID"])&(merge_df["HOUR"]==wdf["H"]),join_type="inner")



In [71]:
merge_df_api.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"TRUCK_ID"  |"MONTH"  |"HOUR"  |"DOW"  |"DAY"  |"MENU_TYPE"  |"PUBLIC_HOLIDAY"  |"SUM(ORDER_TOTAL)"  |"LOCATION"    |"CITY"  |"REGION"  |"LAT"               |"LONG"              |"AVG_TEMPERATURE_AIR_2M_F"  |"AVG_TEMPERATURE_HEATINDEX_2M_F"  |"TOT_PRECIPITATION_IN"  |"TOT_SNOWFALL_IN"  |"TOT_SNOWDEPTH_IN"  |"AVG_CLOUD_COVER_TOT_PCT"  |"l_5fa3_DATE"  |"l_5fa3_LOCATION_ID"  |"SUM_DAY_OF_WEEK_AVG_CITY_MENU_TYPE"  |"SUM_PREV_YEAR_MONTH_SALES_CITY_MENU_TYPE"  |"YEAR" 

In [72]:
merge_df_api=merge_df_api.drop('"l_5fa3_DATE"').drop('"H"' ).drop('"r_mqgl_DATE"').drop('"r_mqgl_LOCATION_ID"')
merge_df_api=merge_df_api.withColumnRenamed('"l_5fa3_LOCATION_ID"',"Location_id")

32

In [78]:
decimal.getcontext().prec=2

In [93]:

dropped_df=merge_df_api.na.fill({ 'SUM_PREV_YEAR_MONTH_SALES_CITY_MENU_TYPE':(1554821.8032227783)})

In [94]:
columns_to_drop=["AVG_TEMPERATURE_AIR_2M_F","AVG_TEMPERATURE_HEATINDEX_2M_F","TOT_PRECIPITATION_IN","TOT_SNOWFALL_IN" ,"TOT_SNOWDEPTH_IN" ,"AVG_CLOUD_COVER_TOT_PCT" ]
new_columns  = [c for c in dropped_df.columns if c not in columns_to_drop]
dropped_df = dropped_df.select(*new_columns)

In [96]:
def one_hot_encode_columns(df, column_names):
    encoded_df = df
    
    for column_name in column_names:
        unique_values = [row[column_name] for row in df.select(column_name).distinct().collect()]
        
        # Create a new column for each unique value and perform one-hot encoding
        for value in unique_values:
            encoded_column_name = f"{column_name}_{value}_encoded"
            encoded_df = encoded_df.withColumn(encoded_column_name, F.when(F.col(column_name) == value, 1).otherwise(0))
    
    
    return encoded_df
dropped_df.select("menu_type").distinct().show()

columns = dropped_df.columns

# Iterate over the columns and count the number of unique values
unique_counts = {}
for column in columns:
    unique_counts[column] = dropped_df.select(column).distinct().count()

# Print the unique counts for each column
for column, count in unique_counts.items():
    print(f"Column '{column}': {count}")

------------------
|"MENU_TYPE"     |
------------------
|Mac & Cheese    |
|Sandwiches      |
|Crepes          |
|BBQ             |
|Gyros           |
|Ice Cream       |
|Chinese         |
|Grilled Cheese  |
|Hot Dogs        |
|Ramen           |
------------------

Column 'TRUCK_ID': 75
Column 'MONTH': 12
Column 'HOUR': 16
Column 'DOW': 7
Column 'DAY': 31
Column 'MENU_TYPE': 15
Column 'PUBLIC_HOLIDAY': 2
Column '"SUM(ORDER_TOTAL)"': 48051
Column 'LOCATION': 1848
Column 'CITY': 5
Column 'REGION': 5
Column 'LAT': 1878
Column 'LONG': 1878
Column 'LOCATION_ID': 1878
Column 'SUM_DAY_OF_WEEK_AVG_CITY_MENU_TYPE': 79664
Column 'SUM_PREV_YEAR_MONTH_SALES_CITY_MENU_TYPE': 1637
Column 'YEAR': 3
Column 'WEATHERCODE': 13


In [97]:
categoricalColumns = ['MENU_TYPE', 'CITY', 'REGION']
encoded_df = one_hot_encode_columns(dropped_df, categoricalColumns)
final_df = encoded_df.drop(*categoricalColumns)
final_df.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"TRUCK_ID"  |"MONTH"  |"H

In [104]:
# final_df=final_df.to_pandas()
final_df=final_df.drop("LOCATION",axis=1)

In [109]:
columns_not_to_scale=["TRUCK_ID","MONTH","HOUR","DOW","DAY","PUBLIC_HOLIDAY","LOCATION_ID","LAT","LONG","SUM(ORDER_TOTAL)","YEAR"]     

cat_col = [i for i in final_df.columns if i not in columns_not_to_scale and  (i[:4] not in ["CITY","REGI","MENU"])]

cat_col.remove("WEATHERCODE")
from sklearn.preprocessing import StandardScaler
X_final_scaled=final_df.copy()
X_final_scaled[cat_col] = StandardScaler().fit_transform(final_df[cat_col])

In [110]:
X_final_scaled["YEAR"]

0         2020
1         2020
2         2020
3         2020
4         2020
          ... 
540196    2020
540197    2020
540198    2020
540199    2020
540200    2020
Name: YEAR, Length: 540201, dtype: int16

In [14]:
from xgboost import XGBRegressor

In [14]:
algo_df = session.table('ROUTING."ALGO_DATA(With Year)"')

In [18]:
algo_df.select("TRUCK_ID").distinct().count()

75

In [19]:
model = joblib.load(open('model.joblib',"rb"))

EOFError: 